In [1]:
import pandas as pd
import numpy as np

In [11]:
res2015 = pd.read_excel("2015 Constituency Results.xlsx")
res2015.rename(columns={"Unnamed: 1": "Name"}, inplace=True)
res2015.head()

,Constituency,Name,Country,Region,Electorate,Turnout,Unnamed: 6,Candidate,Party,Votes,Share,Place
0,1,Aberavon,Wales,Wales,49821,31523,0.633,Stephen Kinnock,Labour,15416,0.489,1
1,1,Aberavon,Wales,Wales,49821,31523,0.633,Peter Bush,UKIP,4971,0.158,2
2,1,Aberavon,Wales,Wales,49821,31523,0.633,Edward Yi He,Conservative,3742,0.119,3
3,1,Aberavon,Wales,Wales,49821,31523,0.633,Duncan Higgitt,Plaid Cymru,3663,0.116,4
4,1,Aberavon,Wales,Wales,49821,31523,0.633,Helen Clarke,Liberal Democrat,1397,0.044,5


# 2010

Can't find a clean list.  But we have:

- https://www.electoralcommission.org.uk/find-information-by-subject/elections-and-referendums/past-elections-and-referendums/uk-general-elections/2010-uk-general-election-results
 
- http://www.politicsresources.net/area/uk/ge10/ge10index.htm


In [11]:
import requests
from bs4 import BeautifulSoup

In [6]:
ec_url = ("https://www.electoralcommission.org.uk/find-information-by-subject/" +
        "elections-and-referendums/past-elections-and-referendums/"+
        "uk-general-elections/2010-uk-general-election-results")

In [9]:
response = requests.get(ec_url)
response

<Response [200]>

In [38]:
soup = BeautifulSoup(response.content, 'html.parser')
for form in soup.find_all("form"):
    if "id" in form.attrs:
        break
con_lookup = {}
for v in form.find_all("option"):
    url = v["value"]
    if url != "":
        con_lookup[v.text] = url

In [5]:
import collections, time

Result = collections.namedtuple("Result", ["name", "party", "votes"])
Constituency = collections.namedtuple("Constituency", ["name", "electorate", "total_votes", "results"])

In [71]:
def get_all(con_lookup):
    for name, url in con_lookup.items():
        time.sleep(1)
        response = requests.get(url)
        assert response.ok
        soup = BeautifulSoup(response.content, 'html.parser')

        name = soup.find_all(attrs={"class":"page-title"})[0].text
        data = [para.text for para in soup.find_all(attrs={"class":"election-results"})[0].find_all("p")]
        for d in data:
            if d.startswith("Electorate: "):
                electorate = int(d[12:].replace(",", ""))
            if d.startswith("Total number of votes cast: "):
                total_votes = int(d[28:].replace(",", ""))
        results = []
        for row in soup.find_all("table")[0].tbody.find_all("tr"):
            entries = [e.text for e in row.find_all("td")]
            v = int(entries[3].replace(",", ""))
            results.append( Result(name=entries[1] + " " +entries[0], party=entries[2], votes=v) )

        yield Constituency(name, electorate, total_votes, results)

In [72]:
results = list(get_all(con_lookup))

In [ ]:
import pickle
with open("results.pic", "wb") as f:
    pickle.dump(f, results)

# One we prepared earlier

In [6]:
import pickle
with open("results.pic", "rb") as f:
    results = pickle.load(f)

In [8]:
results[0]

Constituency(name='Aberavon', electorate=50838, total_votes=31469, results=[Result(name='H. Francis', party='Labour Party', votes=16073), Result(name='K. Davies', party='Liberal Democrat', votes=5034), Result(name='C.Y. Jones', party='Conservative Party', votes=4411), Result(name='P. Nicholls-Jones', party='Plaid Cymru', votes=2198), Result(name='K.A. Edwards', party='British National Party', votes=1276), Result(name='A. Tutton', party='Independent', votes=919), Result(name='C. Beany', party='New Millennium Bean', votes=558), Result(name='J. Callan', party='United Kingdom Independence Party', votes=489)])

In [30]:
data = []
for result in results:
    for votes in result.results:
        data.append([result.name, result.electorate, result.total_votes,
                    votes.name, votes.party, votes.votes])

    all_votes = np.asarray([r.votes for r in result.results])
    places = np.argsort(all_votes)
    length = len(result.results)
    for i, (p,v) in enumerate(zip(places, result.results)):
        data[i-length].append(v.votes / result.total_votes)
        data[i-length].append(length - p)
        
df = pd.DataFrame(data)
df.columns = ["Name", "Electorate", "Turnout", "Candidate", "Party", "Votes", "Share", "Place"]

In [34]:
name_to_number = {row["Name"] : row["Constituency"] for _, row in res2015.iterrows()}
name_to_country = {row["Name"] : row["Country"] for _, row in res2015.iterrows()}
name_to_region = {row["Name"] : row["Region"] for _, row in res2015.iterrows()}

In [51]:
different_names = {
    'Antrim East' : 'East Antrim',
    'Antrim North' : 'North Antrim',
    'Antrim South' : 'South Antrim',
    'Ashton Under Lyne' : 'Ashton-under-Lyne',
    'Down North' : 'North Down',
    'Down South' : 'South Down',
    'Dunfermline & Fife West' : 'Dunfermline & West Fife',
    'Faversham & Kent Mid' : 'Faversham & Mid Kent',
    'Londonderry East' : 'East Londonderry',
    'Tyrone West' : 'West Tyrone',
    'Ulster Mid' : 'Mid Ulster'
}

def adjust_name(n):
    if n in different_names:
        return different_names[n]
    return n

df["Name"] = df.Name.map(adjust_name)

In [55]:
df["Constituency"] = df.Name.map(name_to_number)
df["Country"] = df.Name.map(name_to_country)
df["Region"] = df.Name.map(name_to_region)

In [61]:
cols = list(res2015.columns)
del cols[6]
df[cols].head()

,Constituency,Name,Country,Region,Electorate,Turnout,Candidate,Party,Votes,Share,Place
0,1,Aberavon,Wales,Wales,50838,31469,H. Francis,Labour Party,16073,0.510757,1
1,1,Aberavon,Wales,Wales,50838,31469,K. Davies,Liberal Democrat,5034,0.159967,2
2,1,Aberavon,Wales,Wales,50838,31469,C.Y. Jones,Conservative Party,4411,0.140170,3
3,1,Aberavon,Wales,Wales,50838,31469,P. Nicholls-Jones,Plaid Cymru,2198,0.069847,4
4,1,Aberavon,Wales,Wales,50838,31469,K.A. Edwards,British National Party,1276,0.040548,5


In [62]:
res2015.head()

,Constituency,Name,Country,Region,Electorate,Turnout,Unnamed: 6,Candidate,Party,Votes,Share,Place
0,1,Aberavon,Wales,Wales,49821,31523,0.633,Stephen Kinnock,Labour,15416,0.489,1
1,1,Aberavon,Wales,Wales,49821,31523,0.633,Peter Bush,UKIP,4971,0.158,2
2,1,Aberavon,Wales,Wales,49821,31523,0.633,Edward Yi He,Conservative,3742,0.119,3
3,1,Aberavon,Wales,Wales,49821,31523,0.633,Duncan Higgitt,Plaid Cymru,3663,0.116,4
4,1,Aberavon,Wales,Wales,49821,31523,0.633,Helen Clarke,Liberal Democrat,1397,0.044,5
